# Feature Engineering

## Objective
The goal of this notebook is to create derived features that better capture credit risk by incorporating domain intuition and insights from exploratory data analysis.

In particular, this notebook focuses on:
- Ratio-based features to reflect repayment burden
- Transformations of raw variables into more interpretable forms
- Missingness indicators for high-signal features
- Preparation of a clean modeling dataset

In [ ]:
import pandas as pd
import numpy as np

from pathlib import Path

# Notebook-safe project root resolution
PROJECT_ROOT = Path.cwd().parents[0]
DATA_DIR = PROJECT_ROOT / "data"

pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", "{:.3f}".format)

In [2]:
df = pd.read_csv(DATA_DIR / "application_train.csv")

print("Shape:", df.shape)
df.head()

Shape: (307511, 122)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.000,406597.500,24700.500,351000.000,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.019,-9461,-637,-3648.000,-2120,NaN,1,1,0,1,1,0,Laborers,1.000,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083,0.263,0.139,0.025,0.037,0.972,0.619,0.014,0.000,0.069,0.083,0.125,0.037,0.020,0.019,0.000,0.000,0.025,0.038,0.972,0.634,0.014,0.000,0.069,0.083,0.125,0.038,0.022,0.020,0.000,0.000,0.025,0.037,0.972,0.624,0.014,0.000,0.069,0.083,0.125,0.037,0.021,0.019,0.000,0.000,reg oper account,block of flats,0.015,"Stone, brick",No,2.000,2.000,2.000,2.000,-1134.000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0.000,0.000,0.000,0.000,1.000
1,100003,0,Cash loans,F,N,N,0,270000.000,1293502.500,35698.500,1129500.000,Family,State servant,Higher education,Married,House / apartment,0.004,-16765,-1188,-1186.000,-291,NaN,1,1,0,1,1,0,Core staff,2.000,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311,0.622,NaN,0.096,0.053,0.985,0.796,0.060,0.080,0.035,0.292,0.333,0.013,0.077,0.055,0.004,0.010,0.092,0.054,0.985,0.804,0.050,0.081,0.035,0.292,0.333,0.013,0.079,0.055,0.000,0.000,0.097,0.053,0.985,0.799,0.061,0.080,0.035,0.292,0.333,0.013,0.079,0.056,0.004,0.010,reg oper account,block of flats,0.071,Block,No,1.000,0.000,1.000,0.000,-828.000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0.000,0.000,0.000,0.000,0.000
2,100004,0,Revolving loans,M,Y,Y,0,67500.000,135000.000,6750.000,135000.000,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010,-19046,-225,-4260.000,-2531,26.000,1,1,1,1,1,0,Laborers,1.000,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.556,0.730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [3]:
df["AGE_YEARS"] = (-df["DAYS_BIRTH"] / 365).round(1)

df[["DAYS_BIRTH", "AGE_YEARS"]].head()

,DAYS_BIRTH,AGE_YEARS
0,-9461,25.900
1,-16765,45.900
2,-19046,52.200
3,-19005,52.100
4,-19932,54.600


In [4]:
df["CREDIT_INCOME_RATIO"] = (
    df["AMT_CREDIT"] / df["AMT_INCOME_TOTAL"]
)

df["CREDIT_INCOME_RATIO"].describe()

count   307511.000
mean         3.958
std          2.690
min          0.005
25%          2.019
50%          3.265
75%          5.160
max         84.737
Name: CREDIT_INCOME_RATIO, dtype: float64

In [5]:
df.groupby("TARGET")["CREDIT_INCOME_RATIO"].median()

TARGET
0   3.267
1   3.253
Name: CREDIT_INCOME_RATIO, dtype: float64

In [6]:
df["CIR_BIN"] = pd.qcut(
    df["CREDIT_INCOME_RATIO"],
    q=10,
    duplicates="drop"
)

df.groupby("CIR_BIN")["TARGET"].mean()

C:\Users\jakey\AppData\Local\Temp\ipykernel_11548\3182568767.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("CIR_BIN")["TARGET"].mean()


CIR_BIN
(0.00381, 1.332]   0.069
(1.332, 1.818]     0.078
(1.818, 2.266]     0.081
(2.266, 2.764]     0.090
(2.764, 3.265]     0.087
(3.265, 3.906]     0.092
(3.906, 4.725]     0.086
(4.725, 5.769]     0.080
(5.769, 7.488]     0.074
(7.488, 84.737]    0.071
Name: TARGET, dtype: float64

### Engineered Feature: Credit-to-Income Ratio

- Raw income and credit amounts showed weak individual signal in EDA
- The credit-to-income ratio was hypothesized to better reflect repayment burden
- Analysis reveals a non-monotonic relationship with default risk:
  - Default rates increase from low to mid-range ratios
  - Risk peaks in the middle and declines at very high ratios
- This suggests potential selection effects or nonlinear dynamics in loan approvals
- The feature may be more suitable for tree-based models or binned representations than for linear models

In [7]:
df["ANNUITY_INCOME_RATIO"] = (
    df["AMT_ANNUITY"] / df["AMT_INCOME_TOTAL"]
)

df.groupby("TARGET")["ANNUITY_INCOME_RATIO"].median()

TARGET
0   0.162
1   0.169
Name: ANNUITY_INCOME_RATIO, dtype: float64

In [8]:
df["AIR_BIN"] = pd.qcut(
    df["ANNUITY_INCOME_RATIO"],
    q=10,
    duplicates="drop"
)

df.groupby("AIR_BIN")["TARGET"].mean()

C:\Users\jakey\AppData\Local\Temp\ipykernel_11548\1755689667.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("AIR_BIN")["TARGET"].mean()


AIR_BIN
(-0.000776, 0.08]   0.071
(0.08, 0.104]       0.073
(0.104, 0.125]      0.078
(0.125, 0.144]      0.078
(0.144, 0.163]      0.079
(0.163, 0.186]      0.084
(0.186, 0.212]      0.085
(0.212, 0.247]      0.089
(0.247, 0.302]      0.088
(0.302, 1.876]      0.082
Name: TARGET, dtype: float64

### Engineered Feature: Annuity-to-Income Ratio

- The annuity-to-income ratio exhibits a largely monotonic increase in default risk
- Higher monthly payment burden is associated with higher default probability
- Minor attenuation in the extreme upper tail likely reflects selection effects or sparse data
- This feature is well-suited for linear models such as logistic regression
- Compared to total credit exposure, monthly cash-flow burden appears more predictive of default risk

In [16]:
df.drop(columns=["CIR_BIN", "AIR_BIN"], inplace=True)

In [9]:
ext_features = ["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]

for col in ext_features:
    df[f"{col}_MISSING"] = df[col].isnull().astype(int)

df[[f"{col}_MISSING" for col in ext_features]].mean()

EXT_SOURCE_1_MISSING   0.564
EXT_SOURCE_2_MISSING   0.002
EXT_SOURCE_3_MISSING   0.198
dtype: float64

In [12]:
df.groupby("EXT_SOURCE_1_MISSING")["TARGET"].mean()

EXT_SOURCE_1_MISSING
0   0.075
1   0.085
Name: TARGET, dtype: float64

In [13]:
df.groupby("EXT_SOURCE_2_MISSING")["TARGET"].mean()

EXT_SOURCE_2_MISSING
0   0.081
1   0.079
Name: TARGET, dtype: float64

In [14]:
df.groupby("EXT_SOURCE_3_MISSING")["TARGET"].mean()

EXT_SOURCE_3_MISSING
0   0.078
1   0.093
Name: TARGET, dtype: float64

### Missingness Indicators
- Missing values in external risk scores are associated with higher default rates
- Missingness likely reflects limited or poor credit history
- Explicit missing indicators allow linear models to capture this signal

In [19]:
BASE_FEATURES = ["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3", "AGE_YEARS", "ANNUITY_INCOME_RATIO"]
EXTENDED_FEATURES = BASE_FEATURES + ["CREDIT_INCOME_RATIO"]